In [1]:
import warnings
warnings.filterwarnings('ignore')
import pyspark
from pyspark.ml.feature import StringIndexer

In [2]:
spark = (
    pyspark.sql.SparkSession.builder
        .appName("Spark ML Research")
        .master("local[4]") # limit executor to 4 cores
        .config("spark.executor.memory", "1g")
        .config("spark.driver.memory", "1g")
        .getOrCreate()
)
spark.conf.set('spark.sql.repl.eagerEval.enabled', True)  # to pretty print pyspark.DataFrame in jupyter

sc = spark.sparkContext
sc.setLogLevel("ERROR")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/02/08 20:51:17 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/02/08 20:51:17 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
df = spark.createDataFrame([('a', 1.0), ('a', 2.0), (float("nan"), 3.0),
                            ('b', 4.0), ('c', 5.0)], ["a", "b"])
df

a,b
a,1.0
a,2.0
NaN,3.0
b,4.0
c,5.0


In [4]:
indexer = StringIndexer()\
    .setInputCol("a")\
    .setOutputCol("aIndex")
indexer.fit(df).transform(df).show()

+---+---+------+
|  a|  b|aIndex|
+---+---+------+
|  a|1.0|   0.0|
|  a|2.0|   0.0|
|NaN|3.0|   1.0|
|  b|4.0|   2.0|
|  c|5.0|   3.0|
+---+---+------+



In [5]:
df = spark.createDataFrame([('a', 1.0), ('a', 2.0), (None, 3.0),
                            ('b', 4.0), ('c', 5.0)], ["a", "b"])
df

a,b
a,1.0
a,2.0
NULL,3.0
b,4.0
c,5.0


In [9]:
indexer = StringIndexer()\
    .setInputCol("a")\
    .setOutputCol("aIndex")
indexed = indexer.fit(df).transform(df)

In [10]:
indexed.show(1)

24/02/08 20:52:37 ERROR Executor: Exception in task 1.0 in stage 24.0 (TID 44)
org.apache.spark.SparkException: [FAILED_EXECUTE_UDF] Failed to execute user defined function (`StringIndexerModel$$Lambda/0x0000007001f6cf40`: (string) => double).
	at org.apache.spark.sql.errors.QueryExecutionErrors$.failedExecuteUserDefinedFunctionError(QueryExecutionErrors.scala:198)
	at org.apache.spark.sql.errors.QueryExecutionErrors.failedExecuteUserDefinedFunctionError(QueryExecutionErrors.scala)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apache.spark.sql.execution.WholeStageCodegenEvaluatorFactory$WholeStageCodegenPartitionEvaluator$$anon$1.hasNext(WholeStageCodegenEvaluatorFactory.scala:43)
	at org.apache.spark.sql.execution.SparkPlan.$anonfun$getByteArrayRdd$1(SparkPlan.scala:388)
	at org.apache.spark.rdd.RDD.$anonfun$ma

Py4JJavaError: An error occurred while calling o216.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 1 in stage 24.0 failed 1 times, most recent failure: Lost task 1.0 in stage 24.0 (TID 44) (192.168.1.21 executor driver): org.apache.spark.SparkException: [FAILED_EXECUTE_UDF] Failed to execute user defined function (`StringIndexerModel$$Lambda/0x0000007001f6cf40`: (string) => double).
	at org.apache.spark.sql.errors.QueryExecutionErrors$.failedExecuteUserDefinedFunctionError(QueryExecutionErrors.scala:198)
	at org.apache.spark.sql.errors.QueryExecutionErrors.failedExecuteUserDefinedFunctionError(QueryExecutionErrors.scala)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apache.spark.sql.execution.WholeStageCodegenEvaluatorFactory$WholeStageCodegenPartitionEvaluator$$anon$1.hasN

In [11]:
df = df.fillna('', subset=['a'])
df

a,b
a,1.0
a,2.0
,3.0
b,4.0
c,5.0


In [12]:
indexer = StringIndexer()\
    .setInputCol("a")\
    .setOutputCol("aIndex")
indexed = indexer.fit(df).transform(df)
indexed.show()

+---+---+------+
|  a|  b|aIndex|
+---+---+------+
|  a|1.0|   0.0|
|  a|2.0|   0.0|
|   |3.0|   1.0|
|  b|4.0|   2.0|
|  c|5.0|   3.0|
+---+---+------+

